In [295]:
import pandas as pd
import numpy as np
import os,re,time
from collections import Counter

In [314]:
data = pd.read_csv('applications.csv')

In [315]:
data.head()

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud
0,1,1/1/16,509998359,XRAAAXUAM,SMTAAXRS,4168 XEMMZ PL 19304,19304,11/3/30,6387900398,1
1,2,1/1/16,615509747,SSXTUJSJM,UTUREERX,8409 ASUZ ST 03563,3563,4/10/21,1069037699,0
2,3,1/1/16,532801671,SZMMUJEZS,EZJEAZ,9782 UMSME LN 42178,42178,9/11/13,8719510343,1
3,4,1/1/16,302334738,EAZSRMZXZ,SMSMJMMT,2687 XRXAX DR 34631,34631,6/26/07,6314026324,1
4,5,1/1/16,737610282,SMRAUMMMZ,MEAXJUX,4775 ETRXZ BLVD 88175,88175,6/26/07,9105580920,0


In [316]:
#data['date'] = pd.to_datetime(data['date'])
data = data.iloc[:,1:]

In [317]:
data.head(10)

,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud
0,1/1/16,509998359,XRAAAXUAM,SMTAAXRS,4168 XEMMZ PL 19304,19304,11/3/30,6387900398,1
1,1/1/16,615509747,SSXTUJSJM,UTUREERX,8409 ASUZ ST 03563,3563,4/10/21,1069037699,0
2,1/1/16,532801671,SZMMUJEZS,EZJEAZ,9782 UMSME LN 42178,42178,9/11/13,8719510343,1
3,1/1/16,302334738,EAZSRMZXZ,SMSMJMMT,2687 XRXAX DR 34631,34631,6/26/07,6314026324,1
4,1/1/16,737610282,SMRAUMMMZ,MEAXJUX,4775 ETRXZ BLVD 88175,88175,6/26/07,9105580920,0
5,1/1/16,915986896,SUXEEAZJX,SZEJSXZU,2713 UJZJ ST 09310,9310,5/16/23,9177343015,1
6,1/1/16,896738279,XSJZEXRZJ,TATMSSJ,8261 TSSJ CT 83503,83503,11/19/72,649792058,1
7,1/1/16,601993774,XJZAUEZTX,USSMTRX,3535 RMSJU RD 95839,95839,10/17/95,480945554,0
8,1/1/16,131340674,TZERZRXZ,USZMSMEZ,3307 SUZXR ST 04362,4362,3/14/15,350169510,0
9,1/1/16,888484341,EAXRRUMUX,RAUZRMEA,508 UMJXM BLVD 67490,67490,6/28/86,1557783849,0


In [320]:
data.dtypes

date                  object
ssn                    int64
firstname             object
lastname              object
address               object
zip5                   int64
dob                   object
homephone              int64
fraud                  int64
firstname_lastname    object
dtype: object

In [322]:
data['ssn'] = data['ssn'].astype('str')
data['zip5'] = data['zip5'].astype('str')
data['homephone'] = data['homephone'].astype('str')

In [323]:
data['firstname_lastname'] = data['firstname']+'_'+data['lastname']
data['homephone_address'] = data['homephone']+'_'+data['address']
data['homephone_zip5'] = data['homephone']+'_'+data['zip5']
data['address_zip5'] = data['address']+'_'+data['zip5']
data['ssn_address'] = data['ssn']+'_'+data['address']
data['ssn_dob'] = data['ssn']+'_'+data['dob']
data['ssn_zip5'] = data['ssn']+'_'+data['zip5']
data['ssn_homephone'] = data['ssn']+'_'+data['homephone']

In [326]:
#subset = data.iloc[90000:,:]
subset = data

In [44]:
dates = data['date'].unique()

In [324]:
features = ['ssn','zip5','dob','address','firstname_lastname',
           'homephone_address','homephone_zip5','address_zip5',
           'ssn_address','ssn_dob','ssn_zip5','ssn_homephone']

In [327]:
start = time.time()
# 3 days
dic = {}
ssn_2 = np.array([])
for i in range(len(dates)):
#for i in range(6):
    for f in features:
        if i == 0:
            temp = subset[subset['date']==dates[i]]       
            dic['counter_'+f] = Counter(temp[f])
            dic['temp_count_'+f] = subset[subset['date']==dates[i]][f].apply(lambda x: dic['counter_'+f][x])
            dic[f+'_2'] = dic['temp_count_'+f]
        #counter_ssn = Counter(temp['ssn'])
        #temp_count = subset[subset['date']==dates[i]]['ssn'].apply(lambda x: counter[x])
        #ssn_2 = np.hstack((ssn_2,temp_count))
        elif i == 1:
            temp = subset[(subset['date']==dates[i])|(subset['date']==dates[i-1])]     
            dic['counter_'+f] = Counter(temp[f])
            dic['temp_count_'+f] = subset[subset['date']==dates[i]][f].apply(lambda x: dic['counter_'+f][x])
            dic[f+'_2'] = dic['temp_count_'+f]
        else:
            temp = subset[(subset['date']==dates[i])|(subset['date']==dates[i-1])|(subset['date']==dates[i-2])]
            dic['counter_'+f] = Counter(temp[f])
            dic['temp_count_'+f] = subset[subset['date']==dates[i]][f].apply(lambda x: dic['counter_'+f][x])
            dic[f+'_2'] = np.hstack((dic[f+'_2'],dic['temp_count_'+f]))
        #counter = Counter(temp['ssn'])
        #temp_count = subset[subset['date']==dates[i]]['ssn'].apply(lambda x: counter[x])
        #ssn_2 = np.hstack((ssn_2,temp_count))
end = time.time()
print('time elapsed: {}'.format(str(end-start)))

time elapsed: 187.46360301971436


In [328]:
pd.DataFrame(np.array([dic[f+'_2'] for f in features]).T,columns=[f+'_2' for f in features])

,ssn_2,zip5_2,dob_2,address_2,firstname_lastname_2,homephone_address_2,homephone_zip5_2,address_zip5_2,ssn_address_2,ssn_dob_2,ssn_zip5_2,ssn_homephone_2
0,1,1,1,1,2,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,1,1
5,1,1,1,1,1,1,1,1,1,1,1,1
6,1,1,45,1,1,1,1,1,1,1,1,1
7,1,1,1,1,1,1,1,1,1,1,1,1
8,1,1,1,1,1,1,1,1,1,1,1,1
9,1,1,1,1,1,1,1,1,1,1,1,1
